In [ ]:
import os
import sys
import napari
import numpy as np
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
from tifffile import imread as tiff_read
from tifffile import imwrite as tiff_write
from dask_image.imread import imread as dask_read
import dask.array as da
from dask import distributed
import time
from yaspin import yaspin
from dask.distributed import Client
# import dask delayed
from dask.delayed import delayed

In [ ]:
file_path = '/Volumes/bigData/kkpo_test/mp_test'
dask_max_path = '/Volumes/bigData/kkpo_test/mp_test/dask'
dask_vol_path = '/Volumes/bigData/kkpo_test/mp_test/dask_vol.zarr'
trad_max_path = '/Volumes/bigData/kkpo_test/mp_test/trad'
trad_vol_path = '/Volumes/bigData/kkpo_test/mp_test/trad_vol'

for path in [dask_max_path, dask_vol_path, trad_max_path, trad_vol_path]:
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
files = [file for file in os.listdir(file_path) if file.endswith('.tif') and not file.startswith('.')]
files.sort()
files

#### Current Kkpo version

In [ ]:
%%time

ims = dask_read(file_path + '/' + '*.tif')
print(f'gathered {len(ims)} images and assembled them into a dask array of shape {ims.shape} with chunk size {ims.chunks}')

# save volume as zarr
with yaspin() as sp:
    sp.text = f'Converting full volume to zarr, please be patient...'
    start = time.time()
    da.to_zarr(ims[:,:,::8,::8], dask_vol_path, overwrite=True)
    end = time.time()
    print(f'Saved channel zarr in {round(end - start, 3)} seconds')

# save max projection
with tqdm(total=len(files)) as max_pbar:
    max_pbar.set_description('Saving max projections')
    for tp, tp_name in enumerate(['t000000', 't000001', 't000002', 't000003', 't000004']):
        tiff_write(dask_max_path + '/' + f'{tp_name}_Max.tiff', np.max(ims[tp,:,:,:], axis=0))
        max_pbar.update(1)

In [10]:
from interact.kkpo import Kkpo

file_path = '/Volumes/bigData/kkpo_testing/1_region_2i'

kkpo = Kkpo(file_path)

Left off here:

I need a better way to remove partial chunks. It may be possible to find the first time point (i.e, all the regions, channels, illuminations, etc), and then remove all the other time points that are not that first time point.

In [2]:
from interact.kkpo import Kkpo

file_path = '/Volumes/bigData/kkpo_testing/1_region_2i'

kkpo = Kkpo(file_path)
"""  
 'file_path',
 'files',
 'get_region_info',
 'metadata',
 'objmag',
 'region_names',
 'save_regions',
 'view_volumes'
"""
#print(kkpo.region_names[0])
#interval, timepoint_names, channel_names, illum_names, plane_names = kkpo.get_region_info(kkpo.region_names[0])
#print(interval, timepoint_names, channel_names, illum_names, plane_names)
kkpo.save_regions(overwrite = True, save_vol = True)

Saving region 1/1
num time points = 12
/Volumes/bigData/kkpo_testing/1_region_2i/R0000_processed already exists. Overwriting file!
Saving channel 1/3
Multiple time points with two-sided illumination detected
⠇ Fusing two-sided illumination and converting full volume for C00 to zarr, please be patient...Saved channel C00 in 51.682 seconds


Saving max projections: 100%|██████████| 12/12 [00:28<00:00,  2.36s/it]


Saving channel 2/3
Multiple time points with two-sided illumination detected
⠦ Fusing two-sided illumination and converting full volume for C01 to zarr, please be patient...Saved channel C01 in 49.991 seconds


Saving max projections:  92%|█████████▏| 11/12 [00:26<00:02,  2.39s/it]


IndexError: Index is not smaller than dimension 11 >= 11

In [1]:

from dask_image.imread import imread as dask_read
from pathlib import Path
import numpy as np
file_path = Path('/Volumes/bigData/kkpo_testing/song')
region_name = 'R0000'
ch_name = 'C01'
channel_array = dask_read(file_path /  f'*{region_name}*{ch_name}*.tif')
channel_array


dask.array<_map_read_frame, shape=(12, 192, 2048, 2048), dtype=uint16, chunksize=(1, 192, 2048, 2048), chunktype=numpy.ndarray>

In [2]:
channel_array.chunksize

(1, 192, 2048, 2048)

In [3]:
m = np.max(channel_array[0], axis = 0)
m

dask.array<amax-aggregate, shape=(2048, 2048), dtype=uint16, chunksize=(2048, 2048), chunktype=numpy.ndarray>

In [4]:
m.chunksize

(2048, 2048)

In [6]:
x = m.compute_chunk_sizes()

ValueError: Array chunk size or shape is unknown. Possible solution with x.compute_chunk_sizes()

In [3]:
channel_array.compute_chunk_sizes()

ValueError: Array chunk size or shape is unknown. Possible solution with x.compute_chunk_sizes()

In [ ]:
import dask.array as da
m = da.max(channel_array[0], axis=0)
m

In [ ]:
from tifffile import imwrite
imwrite('/Users/bementmbp/Desktop/Max.tiff', m)

In [ ]:
from interact.kkpo import Kkpo
file_path = '/Volumes/bigData/kkpo_testing/1_region_2i_mini'
kkpo = Kkpo(file_path)
kkpo.view_volumes('R0000')

In [ ]:
from dask_image.imread import imread as dask_read
from pathlib import Path
file_path = Path('/Volumes/bigData/kkpo_testing/song')
region_name = 'R0000'
ch_name = 'C01'
left_illum = 'I0'
right_illum = 'I1'
plane_name = 'P00192'

channel_array_left = dask_read(file_path / f'*{region_name}*{ch_name}_I0*.tif')
#channel_array_right = dask_read(file_path / f'S000_t000000_V000_{region_name}_X000_Y000_{ch_name}_I1_D0_{plane_name}.tif')
channel_array_left

In [ ]:
import dask.array as da
tp=0
tp_name = 0
m = da.max(channel_array_left[tp], axis=0)
m

In [ ]:
from tifffile import imwrite as tiff_write
from pathlib import Path
region_save_path = Path('/Users/bementmbp/Desktop')

tiff_write(region_save_path / f'{region_name}_{ch_name}_{tp_name}_Max.tiff', m)

In [ ]:
import numpy as np
from pathlib import Path
from tifffile import imwrite as tiff_write
import dask.array as da

# create a random dask array with shape (12, 192, 2048, 2048)
r_arr = da.random.random((12, 192, 2048, 2048), chunks=(1, 1, 2048, 2048))
r_arr

In [ ]:
m = da.max(r_arr[0], axis=0)
m

In [ ]:
type(m)

In [ ]:
m.compute_chunk_sizes()

In [ ]:
tiff_write('/Users/bementmbp/Desktop/test.tiff', m)

In [1]:
import dask.array.image

#### same thing but with distributed client

In [ ]:
%%time

# make client with 16 processes
client = Client(n_workers=8)
print(client)

ims = dask_read(file_path + '/' + '*.tif')
print(f'gathered {len(ims)} images and assembled them into a dask array of shape {ims.shape} with chunk size {ims.chunks}')

# save volume as zarr
with yaspin() as sp:
    sp.text = f'Converting full volume to zarr, please be patient...'
    start = time.time()
    da.to_zarr(ims[:,:,::8,::8], dask_vol_path, overwrite=True)
    end = time.time()
    print(f'Saved channel zarr in {round(end - start, 3)} seconds')

# save max projection
with tqdm(total=len(files)) as max_pbar:
    max_pbar.set_description('Saving max projections')
    for tp, tp_name in enumerate(['t000000', 't000001', 't000002', 't000003', 't000004']):
        tiff_write(dask_max_path + '/' + f'{tp_name}_Max.tiff', np.max(ims[tp,:,:,:], axis=0))
        max_pbar.update(1)

#### with client and explicit parallelization...

In [ ]:
%%time
import skimage

# make client with 16 processes
client = Client()
print(client)

ims = dask_read(file_path + '/' + '*.tif')
print(f'gathered {len(ims)} images and assembled them into a dask array of shape {ims.shape} with chunk size {ims.chunks}')

#for tp, tp_name in enumerate(['t000000', 't000001', 't000002', 't000003', 't000004']):
max = delayed(np.max(ims[:,:,:,:], axis=0))
    #save = delayed(tiff_write(dask_max_path + '/' + f'{tp_name}_Max.tiff', max))

def save_file(arr, block_info=None):
    """ Save file to foo-x-y.tif, where x and y are block locations """
    filename = "foo-" + "-".join(map(str, block_info[0]["chunk-location"])) + ".tif"
    skimage.io.imsave(filename, arr)
    return arr

s = delayed(max.map_blocks(save_file, dtype=max.dtype))#.compute())       # call function on every block

max.visualize()
s.visualize()
s.compute()